In [1]:
# Librerías necesarias

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

from glob import glob
from tensorflow.keras.applications import InceptionV3, MobileNetV2, ResNet152, VGG16, VGG19
import sklearn.metrics as sk

# Para descargar en Descargas los archivos mediante files.download('/content/blablabla.txt')
from google.colab import files

# Parámetros de preprocesamiento de imágenes

image_size = (224, 224)  # Tamaño deseado de las imágenes
batch_size = 32          # Tamaño del lote para el entrenamiento

In [2]:
!git clone https://github.com/JuliaQuero/img_dataset_parte1

Cloning into 'img_dataset_parte1'...
remote: Enumerating objects: 95877, done.
remote: Counting objects: 100% (30469/30469), done.
remote: Compressing objects: 100% (29763/29763), done.
remote: Total 95877 (delta 860), reused 30315 (delta 706), pack-reused 65408
Receiving objects: 100% (95877/95877), 135.58 MiB | 19.04 MiB/s, done.
Resolving deltas: 100% (1137/1137), done.
Updating files: 100% (327920/327920), done.


In [ ]:
#for folder in [6,7,9,10,11,12,13,14,15,16,17,18,19,20,21]:
for folder in range(22,29,1):
#for folder in range(37,50,1):
  # Rutas de las imágenes de entrenamiento y prueba
  train_dir = 'img_dataset_parte1/V' + str(folder) + '/train'
  test_dir = 'img_dataset_parte1/V' + str(folder) + '/val'
  # Para obtener las clases de salida
  classes = glob('img_dataset_parte1/V' + str(folder) + '/train/*')
  num_classes = len(classes)

  # Generadores de datos para cargar y preprocesar las imágenes de entrenamiento y prueba
  train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Normaliza los valores de píxeles entre 0 y 1
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

  test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

  # Creo txt
  out = open('V'+str(folder)+'InceptionV3.txt','w')

  # Definimos el modelo
  base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor = None, input_shape=(224, 224, 3))
  # Transfer Learning
  # Freeze convolutinal layers
  base_model.trainable=False
  x = base_model.output
  x = GlobalAveragePooling2D()(x)

  top = Dense(units=num_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=top)

  # Calculo la media de 3 f1score
  f1score = 0.0

  for i in range(1,4,1):
    # Compilación y entrenamiento del modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, epochs=10, validation_data=test_generator)

    # Obtenemos medidas
    Y_pred = model.predict(test_generator)
    y_pred = np.argmax(Y_pred, axis=1)
    y_true = test_generator.classes
    report = sk.classification_report(y_true, y_pred, output_dict=True)
    f1score += report['weighted avg']['f1-score']/3

  # Escribo en txt
  out.write(str(f1score)+'\n')

  # Guardo y descargo en local el txt
  out.close()
  files.download('/content/V'+str(folder)+'InceptionV3.txt')

Found 13114 images belonging to 5 classes.
Found 3281 images belonging to 5 classes.
87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/10
142/410 [=========>....................] - ETA: 15:43 - loss: 1.0079 - accuracy: 0.6673